<a href="https://colab.research.google.com/github/NeagrTavakol/Final_dataminning/blob/main/Final_projet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
#mount drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


read dev dataset

In [4]:
dev_csv="/content/drive/MyDrive/Colab Notebooks/dev.csv"
dev_df = pd.read_csv(dev_csv,sep='\t')
dev_df=dev_df.drop("Unnamed: 0",axis=1)
dev_columns=dev_df.columns

read train dataset

In [5]:
train_csv="/content/drive/MyDrive/Colab Notebooks/train.csv"
train_df = pd.read_csv(train_csv,sep='\t')
train_df=train_df.drop("Unnamed: 0",axis=1)
train_columns = train_df.columns

## **Tokenize Data**

**install packages**

In [6]:
# Install; note that the prefix "!" is not needed if you are running in a terminal
!pip install stanza

# Import the package
import stanza

     |████████████████████████████████| 432 kB 10.3 MB/s 
     |████████████████████████████████| 174 kB 38.1 MB/s 
  Created wheel for emoji: filename=emoji-1.6.3-py3-none-any.whl size=170298 sha256=4a074f0b3e557cf16890cfa7c1e1feac6566f471f55c815f34a3f3c632cb4c35
  Stored in directory: /root/.cache/pip/wheels/03/8b/d7/ad579fbef83c287215c0caab60fb0ae0f30c4d7ce5f580eade
Successfully built emoji


In [7]:
# Download an Persiabn model into the default directory
print("Downloading Persian model...")
stanza.download('fa')

2022-01-27 05:41:11 INFO: Downloading default packages for language: fa (Persian)...


2022-01-27 05:41:23 INFO: Finished downloading models and saved to /root/stanza_resources.


**create models**

In [8]:
# Build a Persian pipeline, with all processors by default
print("Building a Persian pipeline...")
fa_nlp = stanza.Pipeline('fa')

2022-01-27 05:41:44 INFO: Loading these models for language: fa (Persian):
| Processor | Package |
-----------------------
| tokenize  | perdt   |
| mwt       | perdt   |
| pos       | perdt   |
| lemma     | perdt   |
| depparse  | perdt   |

2022-01-27 05:41:44 INFO: Use device: cpu
2022-01-27 05:41:44 INFO: Loading: tokenize


Building a Persian pipeline...


2022-01-27 05:41:44 INFO: Loading: mwt
2022-01-27 05:41:44 INFO: Loading: pos
2022-01-27 05:41:45 INFO: Loading: lemma
2022-01-27 05:41:45 INFO: Loading: depparse
2022-01-27 05:41:46 INFO: Done loading processors!


**find tokens of dataframe and save in array**

In [ ]:
# test_token_df['tokenized_sents'] = test_token_df.apply(lambda row: fa_nlp(row['comment']).sentences.words.text, axis=1)
tokenized_array=[]
for i in range(0,len(train_df)):
  comment=train_df.iloc[i]["comment"]
  tokenized_comment = fa_nlp(comment)
  tokens_array=[]
  for i,sent in enumerate(tokenized_comment.sentences):
    for word in sent.words:
      tokens_array.append(word.text)
  tokenized_array.append(tokens_array)
#word.text, word.lemma, word.pos, word.head, word.deprel))

In [ ]:
#find tokens of rest of dataset (process interupted cause of volume of dataset)
# rest_train_df=train_df[52466:]
for i in range(52466,len(train_df)):
  comment=train_df.iloc[i]["comment"]
  tokenized_comment = fa_nlp(comment)
  tokens_array=[]
  for i,sent in enumerate(tokenized_comment.sentences):
    for word in sent.words:
      tokens_array.append(word.text)
  tokenized_array.append(tokens_array)

/usr/local/lib/python3.7/dist-packages/stanza/models/common/beam.py:86: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  prevK = bestScoresId // numWords


**save tokens array to csv file**

In [ ]:
#save tokens to array
import csv
with open('tokens_array.csv', 'w', newline='') as file:
    mywriter = csv.writer(file, delimiter=',')
    mywriter.writerows(tokenized_array)

**convert tokens array to dataframe **

In [ ]:
# convert tokenized_array to dataframe
tokenized_df=pd.DataFrame(tokenized_array)

**save dataframe to csv file**

In [ ]:
#save dataframe 
tokenized_df.to_csv('tokens_df.csv')

In [ ]:
# with open('tokens_array.csv', 'r', newline='') as file:
#   myreader = csv.reader(file, delimiter=',')
#   for rows in myreader:
#     print(rows)

**### another tokenizer**

In [ ]:
# !pip3 install parsivar
# !pip uninstall parsivary

In [ ]:
# for i in tokens_arr[:10]:
#   print(i,len(i),type(i))

In [ ]:
# from parsivar import Normalizer
# from parsivar import Tokenizer
# fa_normalizer = Normalizer()
# fa_tokenizer = Tokenizer()

In [ ]:
# fa_tokens_array=[]
# for i in range(20,40):
#   comment=train_df.iloc[i]["comment"]
#   t=fa_tokenizer.tokenize_words(comment)
#   comment=fa_normalizer.normalize(comment)
#   t=fa_tokenizer.tokenize_words(comment)
#   tokenized_comment = fa_tokenizer.tokenize_words(comment)
#   tokens_array=[]
#   for i in tokenized_comment:
#     tokens_array.append(i)
#   fa_tokens_array.append(tokens_array)

## **Read tokenized dataet**

In [ ]:
#read dataframe csv file
# tokens_dataframe=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/tokens_df.csv")
# tokens_dataframe=tokens_dataframe.drop("Unnamed: 0",axis=1)
# #tokens_dataframe[52000:52050]

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342

In [ ]:
# arr=tokens_dataframe.to_numpy()
# tokens_arr=[]
# for i in arr :
#   sentense=[]
#   for word in i:
#     if type(word) is str:
#       sentense.append(word)
#   tokens_arr.append(sentense)

**read from array csv file (smaller size)**

تو از اینجا ران کن(فایلش رو تلگرام فرستادم برات)

In [9]:
#read array csv file
tokens_array=pd.read_csv("/content/drive/MyDrive/Colab Notebooks/tokens_array.csv",sep='\t',header=None)
# tokens_array.head()

In [11]:
from types import new_class
# from string import punctuation
import re

punctuation=['!','"','#','$','%','&',"'",'(',')','*','+','،','-','.',','"/",':',';','<','=','>','?','@','[',"\\","]","^",'_','`','{','|','}','~']
# tokens_array.to_numpy()
tokens_arr=[]
for i in tokens_array.to_numpy():
  sent=[]
  for j in i[0].split(','):
    if j not in punctuation:
      sent.append(j)
  tokens_arr.append(sent)
  


In [12]:
from gensim.models import Word2Vec

word2vec = Word2Vec(tokens_arr, min_count=2)

In [29]:
vocabulary = word2vec.wv.vocab
print(vocabulary)
vec_vocab = []
for i in tokens_arr[:1][0]:
  print(i)
  v1 = word2vec.wv[i]
  print(type(v1))
  vec_vocab.append(v1)

#    Word2Vec(i)
#    vec_vocab.append(word2vec.wv(i))
print(vec_vocab)




{'واقعا': <gensim.models.keyedvectors.Vocab object at 0x7f92e4a28a90>, 'حیف': <gensim.models.keyedvectors.Vocab object at 0x7f92e5beb510>, 'وقت': <gensim.models.keyedvectors.Vocab object at 0x7f92e4582ad0>, 'که': <gensim.models.keyedvectors.Vocab object at 0x7f92e4582b50>, 'بنویسم': <gensim.models.keyedvectors.Vocab object at 0x7f92f4b70d50>, 'سرویس': <gensim.models.keyedvectors.Vocab object at 0x7f92e4527690>, 'دهیتون': <gensim.models.keyedvectors.Vocab object at 0x7f92e45275d0>, 'شده': <gensim.models.keyedvectors.Vocab object at 0x7f92e4527550>, 'افتضاح': <gensim.models.keyedvectors.Vocab object at 0x7f92e4510310>, 'قرار': <gensim.models.keyedvectors.Vocab object at 0x7f92e55e5b10>, 'بود': <gensim.models.keyedvectors.Vocab object at 0x7f92e45276d0>, '۱': <gensim.models.keyedvectors.Vocab object at 0x7f92e4527750>, 'ساعته': <gensim.models.keyedvectors.Vocab object at 0x7f92e4527790>, 'برسه': <gensim.models.keyedvectors.Vocab object at 0x7f92e4527710>, 'ولی': <gensim.models.keyedvector

In [20]:
v1 = word2vec.wv['وقت']
print(v1)

[-1.2197034e-01 -5.6382620e-01  1.2770091e-01 -4.0112475e-01
  7.7322114e-01 -3.0360090e-02 -1.3867514e+00 -1.2546977e-01
 -2.5692838e-01  1.0169669e+00  4.6977434e-01  9.1655058e-01
  1.5261622e-01  5.2953709e-02 -1.2703375e+00 -2.3463689e-01
 -5.3243721e-01 -5.5398232e-01 -2.4086046e-01 -4.8432657e-01
 -3.0257323e-01 -3.0569386e-01 -1.7867859e-01 -9.2631765e-02
 -6.5255672e-01 -7.3304701e-01 -8.6759609e-01 -1.2399470e+00
 -3.6054355e-01 -3.5417056e-01 -3.7268195e-02  1.0960445e+00
 -1.6739259e-03  3.0105305e-01  3.0238578e-01 -4.3939573e-01
 -2.9561788e-01  1.2166502e+00  2.0336819e+00  5.5245394e-01
  9.4906211e-01  4.5581046e-02  2.5282866e-01  1.2952244e-01
 -1.2460204e+00 -4.6522760e-01 -7.2181001e-02 -1.4260353e-01
 -1.1105579e+00 -1.5302742e-01  7.4270219e-02 -1.1847059e+00
 -1.7827487e-01  3.8057873e-01  7.4027911e-02 -8.1405437e-01
  5.6782866e-01 -1.1506487e+00 -4.6705067e-01 -1.4918407e+00
 -9.4554293e-01 -1.5463290e-03 -5.3764921e-01 -9.5836125e-02
 -4.5210224e-01  6.59175

In [47]:
# import math
# padded_posts = []
# print(len(vec_vocab))
# for post in [vec_vocab]:
#     # print(post)
#     # Pad short posts with alternating min/max
#     print(len(post))
#     if len(post) < 75:
        
#         # Method 1
#         pointwise_min = np.minimum.reduce(post)
#         pointwise_max = np.maximum.reduce(post)
#         padding = [pointwise_max, pointwise_min]
        
#         # Method 2
#         pointwise_avg = np.mean(post)
#         padding = [pointwise_avg]
#         print(pointwise_avg)
#         print(padding)

#         post += padding * math.ceil((55 - len(post) / 2.0))
        
#     # Shorten long posts or those odd number length posts we padded to 51
#     if len(post) > 55:
#         post = post[:55]
    
#     # Add the post to our new list of padded posts
#     padded_posts.append(post)



# # print(len(padded_posts))

## **Preproccessing**

**Analyze Reviews Length**

In [ ]:
import matplotlib.pyplot as plt
reviews_len = [len(x) for x in tokens_arr]
reviews_len=np.array(reviews_len)

In [ ]:
pd.Series(tokens_arr).hist()
plt.show()
pd.Series(tokens_arr).describe()

In [ ]:
print("min:",reviews_len.min(),"max:",reviews_len.max(),"min:",reviews_len.mean(),"len:",len(reviews_len))

min: 3 max: 362 min: 19.003015873015872 len: 56700


In [ ]:
# for i in range(6,29):
print(np.count_nonzero(reviews_len > 75))

689


In [ ]:
cleaned_rokens_arr=[x for x in tokens_arr if len(x)<75]
print(len(cleaned_rokens_arr))

55978


## **Word embedding**

## **Traingin LSTM**

import tensowerflow

In [ ]:
import tensorflow as tf
print(tf.__version__)

2.7.0
